In [1]:
from xgboost import XGBClassifier
from xgboost import DMatrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import scale
from sklearn import datasets
from sklearn import decomposition
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.svm import SVC
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
%pylab inline
from pandas.tools.plotting import scatter_matrix

Populating the interactive namespace from numpy and matplotlib


In [2]:
result = pd.read_csv("/Users/alexegg/Downloads/panel_stores_products_demos_8_11.csv")

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df=result
df.head()

,panid,week,minute,units,outlet,dollars,iri_key,colupc,ou,est_acv,...,race2,race3,microwave,device_type,zipcode,fipscode,market_based_upon_zipcode,iri_geography_number,ext_fact,cnt
0,1116608,1607,9587,2,DK,4.38,642166,11600014690,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
1,1408492,1632,8079,1,DK,1.12,642166,11600015840,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
2,1809632,1584,6998,2,DK,2.72,642166,11600015840,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
3,1809632,1604,6626,4,DK,5.32,642166,11600015990,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4
4,1163584,1615,651,2,DK,3.00,642166,11600016490,DR,1.674999,...,1,1,1,0,1201,25003,204,1,1,4


In [4]:
df.set_index("panid", inplace=True)
df.head()

,week,minute,units,outlet,dollars,iri_key,colupc,ou,est_acv,marketname,...,race2,race3,microwave,device_type,zipcode,fipscode,market_based_upon_zipcode,iri_geography_number,ext_fact,cnt
panid,,,,,,,,,,,,,,,,,,,,,
1116608,1607,9587,2,DK,4.38,642166,11600014690,DR,1.674999,PITTSFIELD,...,1,1,1,0,1201,25003,204,1,1,4
1408492,1632,8079,1,DK,1.12,642166,11600015840,DR,1.674999,PITTSFIELD,...,1,1,1,0,1201,25003,204,1,1,4
1809632,1584,6998,2,DK,2.72,642166,11600015840,DR,1.674999,PITTSFIELD,...,1,1,1,0,1201,25003,204,1,1,4
1809632,1604,6626,4,DK,5.32,642166,11600015990,DR,1.674999,PITTSFIELD,...,1,1,1,0,1201,25003,204,1,1,4
1163584,1615,651,2,DK,3.00,642166,11600016490,DR,1.674999,PITTSFIELD,...,1,1,1,0,1201,25003,204,1,1,4


## Feature Selection

remove the product related columsn b/c that is what i'm training against:

In [5]:
all_columns = set(df.columns)
product_columns = set(['upc2',
 'l1',
 'l2',
 'l3',
 'l4',
 'l5',
 'vol_eq',
 'product_type',
 'events','colupc', 'cnt'])

#these columns are not usefull in predicting what a user will buy -- we only know this AFTER the transaction
#we want to predict before the transaction what will happen
transaction_columns = set(["dollars", "units"])

#some columns are obsolete per documentation
obsolete_columns = set(["hisp_cat", "hispanic_flag", "race2", "market_based_upon_zipcode",
                       "male_smoke", "microwave"])

#redundant columns
redundant_columns = set(["outlet"])

#invalid columns
#panelist_id cannot be used to make prediction
#year may not be usefull for predictions in the future...
invalid_columns = set(["panelist_id"])

useless_columns = set([ "year", "open", "clsd", "marketname", "est_acv", "county", "ext_fact"])

diff = all_columns - product_columns - transaction_columns - obsolete_columns - redundant_columns - invalid_columns - useless_columns

features = df[list(diff)].copy()
features.head()

,fem_smoke,education_level_reached_by_male_hh,type_of_residential_possession,household_head_race,hh_lang,race3,minute_by_day,device_type,family_size,panelist_type,...,hour,weekday_weekend,marital_status,time_of_day_2,day_of_week,time_of_day,combined_pre_tax_income_of_hh,occupation_code_of_household_head,ou,quarter
panid,,,,,,,,,,,,,,,,,,,,,
1116608,NaN,4,1,1,0,1,947,0,1,0,...,16,weekend,2,03_afternoon,07_Sunday,02_during_office_hours,6,9,DR,Q2
1408492,1,5,1,1,0,1,879,0,4,0,...,15,weekend,1,03_afternoon,06_Saturay,02_during_office_hours,11,1,DR,Q4
1809632,0,5,1,1,0,1,1238,0,4,0,...,21,weekday,1,05_night,05_Friday,03_after_office_hours,11,8,DR,Q1
1809632,0,5,1,1,0,1,866,0,4,0,...,15,weekday,1,03_afternoon,05_Friday,02_during_office_hours,11,8,DR,Q2
1163584,0,5,1,1,0,1,651,0,4,0,...,11,weekday,1,02_between_10_and_noon,01_Monday,02_during_office_hours,10,3,DR,Q3


In [6]:
fem_smoke_dist = lambda: np.random.choice([0,1], p=[0.832,0.168])
features.fem_smoke.fillna(fem_smoke_dist(), inplace=True)
features.all_tvs.fillna(-1, inplace=True)
features.cabl_tvs.fillna(-1, inplace=True)

outlet_dist = lambda: np.random.choice(["GK", "GR", "DR"], p=[0.902, .091, 0.04])
features.ou.fillna(outlet_dist, inplace=True)

features.education_level_reached_by_household_head.fillna(99, inplace=True)
features.age_group_applied_to_household_head.fillna(99, inplace=True)
features.combined_pre_tax_income_of_hh.fillna(99, inplace=True)
features.occupation_code_of_household_head.fillna(99, inplace=True)

In [7]:
features.head()

,fem_smoke,education_level_reached_by_male_hh,type_of_residential_possession,household_head_race,hh_lang,race3,minute_by_day,device_type,family_size,panelist_type,...,hour,weekday_weekend,marital_status,time_of_day_2,day_of_week,time_of_day,combined_pre_tax_income_of_hh,occupation_code_of_household_head,ou,quarter
panid,,,,,,,,,,,,,,,,,,,,,
1116608,0,4,1,1,0,1,947,0,1,0,...,16,weekend,2,03_afternoon,07_Sunday,02_during_office_hours,6,9,DR,Q2
1408492,1,5,1,1,0,1,879,0,4,0,...,15,weekend,1,03_afternoon,06_Saturay,02_during_office_hours,11,1,DR,Q4
1809632,0,5,1,1,0,1,1238,0,4,0,...,21,weekday,1,05_night,05_Friday,03_after_office_hours,11,8,DR,Q1
1809632,0,5,1,1,0,1,866,0,4,0,...,15,weekday,1,03_afternoon,05_Friday,02_during_office_hours,11,8,DR,Q2
1163584,0,5,1,1,0,1,651,0,4,0,...,11,weekday,1,02_between_10_and_noon,01_Monday,02_during_office_hours,10,3,DR,Q3


In [8]:
#target encoder
target_column_name="l2"
target_column = df[target_column_name]
le = preprocessing.LabelEncoder()
le.fit(list(set(target_column.values)))
target = le.transform(target_column.values)
list(set(target_column.values))

['TORTILLA/TOSTADA CHIPS',
 'PRETZELS',
 'PORK RINDS',
 'READY-TO-EAT POPCORN/CARAMEL COR',
 'POTATO CHIPS',
 'CHEESE SNACKS',
 'OTHER SALTED SNACKS (NO NUTS)',
 'CORN SNACKS (NO TORTILLA CHIPS)']

In [9]:
features3=pd.get_dummies(features)
features3.shape

(461025, 62)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(features3, target, test_size=0.25, random_state=4)
print (X_train.shape)

(345768, 62)


In [13]:
kbest = SelectKBest(f_classif)
pipeline = Pipeline([('kbest', kbest),('rf', RandomForestClassifier())])
grid_search = GridSearchCV(pipeline, {'kbest__k': [1,3,8,10,16,20]})
grid_search.fit(X_train, Y_train)
print grid_search.best_estimator_
print grid_search.best_params_
print grid_search.score(X_test, Y_test)
#k=16, score: .451

Pipeline(steps=[('kbest', SelectKBest(k=16, score_func=<function f_classif at 0x10eef7a28>)), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
{'kbest__k': 16}
0.44982951144


In [12]:
for params, mean_score, scores in grid_search.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))

0.423 (+/-0.000) for {'kbest__k': 1}
